In [ ]:
import warnings

warnings.filterwarnings("ignore")

import os
import re
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
with open("d23_input.txt", "r") as f:
    lines = f.read().split("\n")
# lines = [(i[0], int(i[1:])) for i in lines]
lines[0], lines[-1]

('685974213', '685974213')

In [3]:
start_li = [int(i) for i in lines[0]]
start_li

[6, 8, 5, 9, 7, 4, 2, 1, 3]

In [4]:
destination_idx = 8
start_li[: destination_idx + 1] + ["t"] + start_li[destination_idx + 1 :]

[6, 8, 5, 9, 7, 4, 2, 1, 3, 't']

In [5]:
class crab_cup:
    """
    use list
    slow version
    """

    def __init__(self, init_li, pick_up_num=3):
        self.cup_li = init_li
        self.pick_up_num = pick_up_num
        self.round = 0

    def play_one_round(self):
        current_cup = self.cup_li[0]
        pick_up_li = self.cup_li[1 : 1 + self.pick_up_num]
        rest_li = self.cup_li[1 + self.pick_up_num :]
        cup_to_find = current_cup - 1
        min_to_find = min(rest_li)
        while cup_to_find in pick_up_li:
            cup_to_find -= 1
            if cup_to_find < min_to_find:
                break
        if cup_to_find < min_to_find:  # find max
            destination_cup = max(rest_li)
        else:
            # find in the rest_li
            destination_cup = cup_to_find
        # get destination_cup's index
        destination_idx = rest_li.index(destination_cup)
        # re-arrange
        self.cup_li = (
            rest_li[: destination_idx + 1]
            + pick_up_li
            + rest_li[destination_idx + 1 :]
            + [current_cup]
        )
        self.round += 1

    def show_current_cup_li(self):
        print(f"The cup list after {self.round} round: {self.cup_li}")

    def play_n_rounds(self, round_num):
        for _ in range(round_num):
            self.play_one_round()

In [6]:
new_game = crab_cup(start_li)
new_game.show_current_cup_li()
new_game.play_n_rounds(100)
new_game.show_current_cup_li()

The cup list after 0 round: [6, 8, 5, 9, 7, 4, 2, 1, 3]
The cup list after 100 round: [1, 8, 2, 6, 3, 5, 9, 4, 7]


In [7]:
def find_output(li):
    one_index = li.index(1)
    output_li = li[one_index + 1 :] + li[:one_index]
    output_li = [str(i) for i in output_li]
    return "".join(output_li)

In [8]:
find_output(new_game.cup_li)

'82635947'

In [9]:
start_li = [int(i) for i in lines[0]]
start_li

[6, 8, 5, 9, 7, 4, 2, 1, 3]

In [10]:
max_num = 1_000_000
start_li = start_li + list(range(max(start_li) + 1, max_num + 1))
print(min(start_li), max(start_li))

1 1000000


In [11]:
# create a dict
# each key has a value: (cup_number_befre, cup_number_after)

cup_dict = {
    cup: (start_li[idx - 1], start_li[(idx + 1) % max_num])
    for idx, cup in enumerate(start_li)
}
cup_dict[6], cup_dict[3], cup_dict[1000], cup_dict[max_num - 1], cup_dict[max_num]

((1000000, 8), (1, 10), (999, 1001), (999998, 1000000), (999999, 6))

In [12]:
class crab_cup:
    """
    use dict
    only update locally
    fast version !!
    """

    def __init__(self, init_dict, start_cup):
        self.cup_dict = init_dict
        self.current_cup = start_cup
        self.max_num = len(init_dict)
        self.round = 0

    def find_next_num(self, c, step):
        for _ in range(step):
            c = self.cup_dict[c][1]
        return c

    def play_one_round(self):
        cup_to_find = self.current_cup - 1
        a, b, c = (self.find_next_num(self.current_cup, i + 1) for i in range(3))
        pick_up_li = [a, b, c]
        while cup_to_find in pick_up_li:
            cup_to_find -= 1
        if cup_to_find == 0:  # find max
            destination_cup = self.max_num
            while destination_cup in pick_up_li + [self.current_cup]:
                destination_cup -= 1
        else:
            destination_cup = cup_to_find

        # prepare for next round
        next_start_cup = self.cup_dict[c][1]
        # cut off pick_up_li
        self.cup_dict[self.current_cup] = (
            self.cup_dict[self.current_cup][0],
            next_start_cup,
        )
        self.cup_dict[next_start_cup] = (
            self.current_cup,
            self.cup_dict[next_start_cup][1],
        )
        # insert pick_up_li
        after_destination_cups = self.cup_dict[destination_cup][1]
        self.cup_dict[destination_cup] = (self.cup_dict[destination_cup][0], a)
        self.cup_dict[a] = (destination_cup, b)
        self.cup_dict[c] = (b, after_destination_cups)
        self.cup_dict[after_destination_cups] = (
            c,
            self.cup_dict[after_destination_cups][1],
        )

        self.current_cup = next_start_cup
        self.round += 1

    def show_numbers_after(self, current_num, step=1):
        show_li = [current_num]
        for _ in range(step):
            current_num = self.cup_dict[current_num][1]
            show_li.append(current_num)
        print(f"The cup list after {self.round} round:")
        print(f"Cups: {show_li}")
        return show_li

    def play_n_rounds(self, round_num):
        for _ in tqdm(range(round_num)):
            self.play_one_round()

In [13]:
new_game = crab_cup(cup_dict.copy(), start_cup=start_li[0])
new_game.show_numbers_after(1, 10)

The cup list after 0 round:
Cups: [1, 3, 10, 11, 12, 13, 14, 15, 16, 17, 18]


[1, 3, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [14]:
new_game.play_n_rounds(10_000_000)

100%|██████████████████████████████████████████████████| 10000000/10000000 [01:00<00:00, 164409.18it/s]


In [15]:
res_li = new_game.show_numbers_after(1, 3)
res_li

The cup list after 10000000 round:
Cups: [1, 470997, 333437, 305064]


[1, 470997, 333437, 305064]

In [16]:
# output
res_li[1] * res_li[2]

157047826689